In [1]:
import numpy as np
import pandas as pd
from autogluon.tabular import TabularPredictor
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

train_path = './playground-series-s4e8/train.csv'
test_path = './playground-series-s4e8/test.csv'
sub_path = './playground-series-s4e8/sample_submission.csv'

train_df = pd.read_csv(train_path).drop('id', axis=1)
test_df = pd.read_csv(test_path).drop('id', axis=1)
sub = pd.read_csv(sub_path)

In [2]:
predictor = TabularPredictor.load("AutogluonModels/ag-20240830_102213")

In [3]:
predictor.model_names()

['LightGBMXT_BAG_L1',
 'LightGBM_BAG_L1',
 'RandomForestGini_BAG_L1',
 'RandomForestEntr_BAG_L1',
 'CatBoost_BAG_L1',
 'ExtraTreesGini_BAG_L1',
 'ExtraTreesEntr_BAG_L1',
 'NeuralNetFastAI_BAG_L1',
 'XGBoost_BAG_L1',
 'NeuralNetTorch_BAG_L1',
 'LightGBMLarge_BAG_L1',
 'CatBoost_r177_BAG_L1',
 'WeightedEnsemble_L2',
 'LightGBMXT_BAG_L2',
 'LightGBM_BAG_L2',
 'RandomForestGini_BAG_L2',
 'RandomForestEntr_BAG_L2',
 'CatBoost_BAG_L2',
 'ExtraTreesGini_BAG_L2',
 'ExtraTreesEntr_BAG_L2',
 'NeuralNetFastAI_BAG_L2',
 'XGBoost_BAG_L2',
 'WeightedEnsemble_L3',
 'LightGBMXT_BAG_L3',
 'LightGBM_BAG_L3',
 'RandomForestGini_BAG_L3',
 'RandomForestEntr_BAG_L3',
 'CatBoost_BAG_L3',
 'ExtraTreesGini_BAG_L3',
 'ExtraTreesEntr_BAG_L3',
 'NeuralNetFastAI_BAG_L3',
 'XGBoost_BAG_L3',
 'LightGBMLarge_BAG_L3',
 'WeightedEnsemble_L4']

In [5]:
result_1 = predictor.predict(test_df, model='WeightedEnsemble_L4')
result_2 = predictor.predict(test_df, model='WeightedEnsemble_L3')
result_3 = predictor.predict(test_df, model='RandomForestEntr_BAG_L2')
result_4 = predictor.predict(test_df, model='ExtraTreesGini_BAG_L3')
result_5 = predictor.predict(test_df, model='ExtraTreesEntr_BAG_L3')


/home/pinoc/automl/.gluonvenv/lib/python3.10/site-packages/torch/cuda/__init__.py:118: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [15]:
sub['class1'] = result_1
sub['class2'] = result_2
sub['class3'] = result_3
sub['class4'] = result_4
sub['class5'] = result_5

In [9]:
sub.head()

,id,class,class1,class2,class3,class4,class5
0,3116945,e,e,e,e,e,e
1,3116946,e,p,p,p,p,p
2,3116947,e,p,p,p,p,p
3,3116948,e,p,p,p,p,p
4,3116949,e,e,e,e,e,e


In [10]:
result_6 = predictor.predict(test_df, model='LightGBM_BAG_L3')
result_7 = predictor.predict(test_df, model='NeuralNetFastAI_BAG_L2')

In [16]:
sub['class6'] = result_6
sub['class7'] = result_7

In [17]:
sub['class1'] = sub['class1'].map({'e':0, 'p':1})
sub['class2'] = sub['class2'].map({'e':0, 'p':1})
sub['class3'] = sub['class3'].map({'e':0, 'p':1})
sub['class4'] = sub['class4'].map({'e':0, 'p':1})
sub['class5'] = sub['class5'].map({'e':0, 'p':1})
sub['class6'] = sub['class6'].map({'e':0, 'p':1})
sub['class7'] = sub['class7'].map({'e':0, 'p':1})

In [18]:
sub.head()

,id,class,class1,class2,class3,class4,class5,class6,class7
0,3116945,e,0,0,0,0,0,0,0
1,3116946,e,1,1,1,1,1,1,1
2,3116947,e,1,1,1,1,1,1,1
3,3116948,e,1,1,1,1,1,1,1
4,3116949,e,0,0,0,0,0,0,0


In [22]:
sub.iloc[:, 1]

0          e
1          e
2          e
3          e
4          e
          ..
2077959    e
2077960    e
2077961    e
2077962    e
2077963    e
Name: class, Length: 2077964, dtype: object

In [41]:
sub['class'] = (sub.iloc[:, 2]+sub.iloc[:, 3]+sub.iloc[:, 4]+sub.iloc[:, 5]+sub.iloc[:, 6])/5

In [42]:
sub['class'].head()

0    0.0
1    1.0
2    1.0
3    1.0
4    0.0
Name: class, dtype: float64

In [43]:
# Create masks for conditions
mask_greater_equal = sub['class'] >= 0.5
mask_less = sub['class'] < 0.5

# Apply modifications
sub.loc[mask_greater_equal, 'class'] = 'p'
sub.loc[mask_less, 'class'] = 'e'

In [44]:
sub['class'].head(10)

0    e
1    p
2    p
3    p
4    e
5    e
6    e
7    p
8    p
9    e
Name: class, dtype: object

In [ ]:
test_result = []
for i in range(0, 5):
    test_result.append(xgb_list[i].predict(test_df))

final = np.average(test_result, axis = 0)
final = np.where(final>=0.5, 'p', 'e')

sub['class'] = final
sub.to_csv('sub_Xgb.csv', index = False)

In [45]:
res = pd.read_csv(sub_path)
# sub.to_csv('submission.csv', index=False)
res['class'] = sub['class']
res.head()

,id,class
0,3116945,e
1,3116946,p
2,3116947,p
3,3116948,p
4,3116949,e


In [46]:
res.to_csv('automl_ensemble5_v4.csv', index=False)